In [5]:
import numpy as np
import pandas as pd
import dtale
import seaborn as sns
# sns.set_palette("Set3")
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Read data

In [6]:
data = pd.read_csv('./data/230517COVID19MEXICO.csv', 
                   na_values=[97, 98, 99, 999, 9999-99-99, '9999-99-99', np.nan, 'non', 'no value'],
                   # importing a sample of the dataset
                   nrows=10000
                  )

In [7]:
data.head(1)

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2023-05-16,0741e4,2,6,9,2,9.0,9,16,2,...,2.0,1,4.0,2,NaN,2,NaN,México,NaN,2.0


In [8]:
data.shape

(10000, 40)

In [9]:
# r library for building charts from Mexico: https://github.com/diegovalle/mxmaps/tree/master
# data.MUNICIPIO_RES.value_counts()[0:30]

In [10]:
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

# Result: medical follow up after hospitalization

In [119]:
# hospitalized > intubado ou UCI > death

import pandas as pd
import numpy as np

def create_patient_status(tipo_paciente, intubado, fecha_def, UCI):
    def categorize_patient(row):
        
        if not pd.isna(row['FECHA_DEF']): #if empty death date, patient dismissed alive
            return "Deceased"
            
        elif row['UCI'] == 1:
            return "Ventilated or UCI"
        
        elif row['TIPO_PACIENTE'] == 2 and row['INTUBADO'] == 2 and row['UCI'] == 2:
            return "Ventilated or UCI"
            
        elif row['TIPO_PACIENTE'] == 2 and row['INTUBADO'] == 2:
            return "Ventilated"

        else:
            return "Dismissed no complications"

    patient_status = pd.DataFrame({'TIPO_PACIENTE': tipo_paciente, 
                                   'INTUBADO': intubado, 
                                   'FECHA_DEF': fecha_def,
                                   'UCI': UCI
                                  })
    patient_status['Patient_Status'] = patient_status.apply(categorize_patient, axis=1)

    return patient_status['Patient_Status']


df = pd.DataFrame(data)

result = create_patient_status(df['TIPO_PACIENTE'], df['INTUBADO'], df['FECHA_DEF'], df['UCI'])

In [120]:
df = data.copy()
df['result'] = result

In [121]:
df.result.value_counts()

Dismissed no complications    9404
Ventilated or UCI              519
Deceased                        77
Name: result, dtype: int64

In [122]:
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,result
0,2023-05-16,0741e4,2,6,9,2,9.0,9,16,2,...,1,4.0,2,NaN,2,NaN,México,NaN,2.0,Deceased
1,2023-05-16,0ed066,2,12,16,2,16.0,16,41,1,...,2,NaN,1,2.0,7,NaN,México,NaN,NaN,Dismissed no complications
2,2023-05-16,08c5f9,1,3,15,1,9.0,15,60,2,...,2,NaN,1,2.0,7,NaN,México,NaN,2.0,Ventilated or UCI
3,2023-05-16,0a1655,2,6,9,2,9.0,9,2,2,...,1,4.0,2,NaN,5,NaN,México,NaN,2.0,Ventilated or UCI
4,2023-05-16,13cf10,2,9,9,1,9.0,9,14,1,...,1,1.0,2,NaN,3,NaN,México,NaN,NaN,Dismissed no complications


In [126]:
df.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI', 'result'],
      dtype='object')

In [127]:
df.RESULTADO_LAB

0       4.0
1       NaN
2       NaN
3       4.0
4       1.0
       ... 
9995    2.0
9996    2.0
9997    2.0
9998    NaN
9999    1.0
Name: RESULTADO_LAB, Length: 10000, dtype: float64

# Crosstabs

In [123]:
pre_conditions = ['DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 
                  'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'OTRA_COM']

demographics = ['death', 'SEXO', 'EDAD', 'EMBARAZO', 'INDIGENA', 'MUNICIPIO_RES', 'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN']

covid = [ 'death', 'TIPO_PACIENTE', 'TOMA_MUESTRA_ANTIGENO', 
         'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL', 'UCI', ]

date = ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF']

In [124]:
df.head(1)

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,result
0,2023-05-16,0741e4,2,6,9,2,9.0,9,16,2,...,1,4.0,2,NaN,2,NaN,México,NaN,2.0,Deceased


In [125]:
data.RESULTADO_LAB.value_counts()

2.0    1358
1.0     940
4.0     140
Name: RESULTADO_LAB, dtype: int64